# Optimization of a weekly replanishments 

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import pandas as pd

## Define model parameters

In [5]:
Days=["Mon", "Wed", "Fri"]
Machines = {"M1": 600, "M2": 500, "M3":80, "M4": 80, "M5":300, "M6":200, "M7":80, "M8":100}
fixed_machines={"M5": "Wed", "M6": "Wed", "M8": "Fri"} 
limit_one_machine=1
limit_day=4

## Initialize model and variables 

In [18]:
model = ConcreteModel()
# binary var representing if machine is scheduled
model.ITEMS = Set(initialize=Machines.keys())
model.replanishments = Var(((m, day) for m in model.ITEMS for day in Days), within=Binary, initialize=1)

## Define the objective function

In [19]:
model.obj = Objective(expr=sum([max(Machines[i]*value(model.replanishments[i, day]) for i in model.ITEMS) + \
                            sum(model.replanishments[i, day]*60 for i in model.ITEMS) for day in Days]) - 60, sense=minimize)


## Add model constraints

In [20]:
model.constraints = ConstraintList()  # Create a set of constraints

for machine in model.ITEMS:
    model.constraints.add(limit_one_machine == sum(model.replanishments[machine, day] for day in Days))
    # one replanishment for machine

for day in Days:
    model.constraints.add(limit_day >= sum(model.replanishments[machine, day] for machine in model.ITEMS))
    #max replanishment for one day

for day in Days:
    model.constraints.add(sum(model.replanishments[machine, day] for machine in model.ITEMS)>=0)       

for machine, day in fixed_machines.items():
    model.constraints.add(1==model.replanishments[machine, day])
    #cons for fixed ATMs

## Find the solution 

### Local solver:

In [23]:
opt = SolverFactory("bonmin")  # choose a solver
results = opt.solve(model, tee=True) # solve the model with the selected solver

Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 2220        8 0.02
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 2220        0 0
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 2220        0 0
Cbc0012I Integer solution of 2220 found by DiveMIPFractional after 0 iterations and 0 nodes (0.01 sec

# Final Schedule

In [26]:
machines=[m for m in Machines.keys()] 
Mon= [value(model.replanishments[i,'Mon']) for i in model.ITEMS]
Wed= [value(model.replanishments[i,'Wed']) for i in model.ITEMS]
Fri= [value(model.replanishments[i,'Fri']) for i in model.ITEMS]
schedule={"Machine": machines, "Mon": Mon, "Wed": Wed, "Fri": Fri}
pd.DataFrame(schedule)

,Machine,Mon,Wed,Fri
0,M1,0.0,0.0,1.0
1,M2,0.0,0.0,1.0
2,M3,0.0,0.0,1.0
3,M4,1.0,0.0,0.0
4,M5,0.0,1.0,0.0
5,M6,0.0,1.0,0.0
6,M7,1.0,0.0,0.0
7,M8,0.0,0.0,1.0
